In [ ]:
# import datasets
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2023-04-07 04:35:44--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 404 The specified resource does not exist.
2023-04-07 04:35:45 ERROR 404: The specified resource does not exist..



In [ ]:
!ls

drive  README_rpc-cv-images.txt  rockpaperscissors.zip	runs	     scissors
paper  rock			 rps-cv-images		sample_data


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


In [3]:
# !kaggle datasets download -d karakaggle/kaggle-cat-vs-dog-dataset
!kaggle datasets download -d drgfreeman/rockpaperscissors


 99% 304M/306M [00:10<00:00, 30.6MB/s]
100% 306M/306M [00:10<00:00, 31.9MB/s]


In [4]:
from zipfile import ZipFile
file_name = "/content/rockpaperscissors.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

  

Done


In [5]:
import os

file_path = 'rps-cv-images/paper/Thumbs.db'
if os.path.exists(file_path):
    os.remove(file_path)

file_path = 'rps-cv-images/rock/Thumbs.db'
if os.path.exists(file_path):
    os.remove(file_path)

file_path = 'rps-cv-images/scissors/Thumbs.db'
if os.path.exists(file_path):
    os.remove(file_path)

file_path = 'content/rps-cv-images/README_rpc-cv-images.txt'
if os.path.exists(file_path):
    os.remove(file_path)
    

In [ ]:
os.path.exists('content/rps-cv-images/README_rpc-cv-images.txt')

In [ ]:
!ls

In [108]:
import torch
import numpy as np
import torch.nn
from PIL import Image
from torchvision import transforms
import os
from sklearn.preprocessing import LabelEncoder
import random
from torch.utils.data import Dataset
from zipfile import ZipFile
class custom_dataset(Dataset):

    # initialize your dataset class
    def __init__(self, mode="train", tr=None, image_path="rps-cv-images", label_path="rps-cv-images"):
        self.mode = mode
        self.image_path = image_path
        self.label_path = label_path
        self.total_images = []
        self.labels = []
        self.papers = []
        self.rocks = []
        self.scissorss = []
        self.tr = tr
        self.unique_labels = []
        

        for i in os.listdir(image_path):
            class_balance = []
            self.unique_labels.append(i)
            for j in os.listdir(image_path+"/" + i):
                class_balance.append(image_path+"/"+i+"/"+j)

                self.labels.append(i)
                self.total_images.append(i+'/'+j)
            if str.lower(i) == 'paper':
                self.papers = class_balance
            elif str.lower(i) == 'rock':
                self.rocks = class_balance
            elif str.lower(i) == 'scissors':
                self.scissorss = class_balance
        print("check if dataset is balanced: ")
        print(len(self.papers), len(self.rocks),len(self.scissorss))
        print(len(self.total_images))
        print(len(self.labels))
        self.unique_labels.remove('.ipynb_checkpoints')
        print(self.unique_labels)
        
        
        # # encode the labels as integers
        # self.label_encoder = LabelEncoder()
        # self.labels = self.label_encoder.fit_transform(self.labels)
        
        np.random.seed(42)

        # Combine the image paths and labels into arrays
        total_images = np.array(self.total_images)
        labels = np.array(self.labels)
        # print(type(total_images))
        # print(type(labels))
        # Generate a random permutation of the indices
        
        indices = np.random.permutation(len(total_images))

        # Shuffle the images and labels arrays using the permuted indices
        shuffled_images = total_images[indices]
        shuffled_labels = labels[indices]
        # Shuffle the images and labels using a random state
        #distribute to val train and test


        train_size = 1908
        val_size = 88
        test_size = 192


        # Split the dataset into training (22,000), testing (2,000), and validation (1,000) sets
        train_images = shuffled_images[:train_size]
        train_labels=shuffled_labels[:train_size]

        test_images = shuffled_images[train_size:train_size+test_size]
        test_labels = shuffled_labels[train_size:train_size+test_size]

        val_images = shuffled_images[train_size+test_size:]
        val_labels = shuffled_labels[train_size+test_size:]



        if(self.mode == "train"):
            self.image_list = train_images
            self.labels = train_labels
        elif(self.mode == "val"):
            self.image_list = val_images
            self.labels = val_labels
        elif(self.mode == "test"):
            self.image_list = test_images
            self.labels = test_labels

        

    def __getitem__(self, index):
        

        image = Image.open(self.image_path+'/'+self.image_list[index])
        label = self.labels[index]
        # print(label)
        # print(self.unique_labels)
        label = self.parse_labels(label)
        # print(label)
        # Convert the image to RGB scale
        image = image.convert('RGB')
        
        # image.show()
          

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        

        # Data augmentation transformations
        augmentation_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
        ])
        # Create a transformation that normalizes the image
        transform = transforms.Compose([
            transforms.Resize(
            (224, 224), interpolation=transforms.InterpolationMode.NEAREST),# Resize the image to 224x224
            transforms.CenterCrop(size=(224, 224)),   
            transforms.ToTensor(),           # Convert the image to a tensor
            transforms.Normalize(mean, std)  # Normalize the image
        ])

        if(self.tr):
            image = self.augmentation_transforms(image)
        
        
        
        image = transform(image)
        

        return image, label
    
    
    def parse_labels(self, label):
        arr = np.zeros((len(self.unique_labels),), dtype= float)
        for i in range(len(self.unique_labels)):
            if label == self.unique_labels[i]:
                arr[i] = 1.0
        return arr
    


    def __len__(self):
        return len(self.image_list)
        # return None


custom_dataset()[10]
# print(self.label)
# cat is 0 and dog is 1


check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']


(tensor([[[-1.4843, -1.5014, -1.5014,  ..., -2.0323, -1.9638, -1.8439],
          [-1.5357, -1.5699, -1.5870,  ..., -2.0152, -1.9980, -1.8953],
          [-1.5357, -1.5699, -1.6042,  ..., -2.0152, -2.0323, -1.9295],
          ...,
          [-1.5699, -1.6042, -1.5699,  ..., -1.6042, -1.6042, -1.5357],
          [-1.5870, -1.6042, -1.5699,  ..., -1.6042, -1.6042, -1.5357],
          [-1.5870, -1.6042, -1.6042,  ..., -1.6042, -1.5699, -1.5357]],
 
         [[ 0.1001,  0.0826,  0.0826,  ..., -1.8431, -1.7731, -1.6506],
          [ 0.1352,  0.1176,  0.1176,  ..., -1.7731, -1.7556, -1.6506],
          [ 0.1352,  0.1176,  0.1001,  ..., -1.7731, -1.8081, -1.6856],
          ...,
          [ 0.1001,  0.1001,  0.1001,  ...,  0.1527,  0.1527,  0.1176],
          [ 0.0826,  0.1001,  0.1001,  ...,  0.1527,  0.1527,  0.1176],
          [ 0.1176,  0.1001,  0.1176,  ...,  0.1527,  0.1352,  0.1176]],
 
         [[-1.0550, -1.0724, -1.0724,  ..., -1.5953, -1.5430, -1.4210],
          [-1.0550, -1.1073,

In [103]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [111]:
import torch
import torch.nn as nn

class ExModel(nn.Module):

    def __init__(self, n_classes):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3, kernel_size=(3, 3), out_channels=64)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.max_pooling1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.conv2 = nn.Conv2d(in_channels=64, kernel_size=(3, 3), out_channels=128)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        self.max_pooling2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.conv3 = nn.Conv2d(in_channels=128, kernel_size=(3, 3), out_channels=256)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU()
        self.max_pooling3 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.conv4 = nn.Conv2d(in_channels=256, kernel_size=(3, 3), out_channels=512)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.ReLU()
        self.max_pooling4 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(p=0.5)
        self.classifier_layer1 = nn.Linear(in_features=73728, out_features=1024)
        self.bn5 = nn.BatchNorm1d(1024)
        self.relu5 = nn.ReLU()

        self.dropout2 = nn.Dropout(p=0.5)
        self.classifier_layer2 = nn.Linear(in_features=1024, out_features=512)
        self.bn6 = nn.BatchNorm1d(512)
        self.relu6 = nn.ReLU()

        self.classifier_layer3 = nn.Linear(in_features=512, out_features=n_classes)

    def forward(self, image):
        x = self.conv1(image)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.max_pooling1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.max_pooling2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.max_pooling3(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.max_pooling4(x)

        x = self.flatten(x)

        x = self.dropout1(x)
        x = self.classifier_layer1(x)
        x = self.bn5(x)
        x = self.relu5(x)

        x = self.dropout2(x)
        x = self.classifier_layer2(x)
        x = self.bn6(x)
        x = self.relu6(x)

        pred = self.classifier_layer3(x)

        return pred


In [9]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 23.5 MB/s eta 0:00:00


In [105]:
from torchmetrics.classification import MulticlassF1Score
import torch


def metrics(preds, target):
    metr = MulticlassF1Score(num_classes=3)

    return metr(preds, target)



In [112]:
import torch
import torch.nn as nn
from torch.nn.functional import softmax
from torch.nn.functional import cross_entropy
from torchmetrics import F1Score


from torch.utils.data import DataLoader, Dataset
# from models.example_model import ExModel
# from datasets.dataset_retrieval import custom_dataset
from torch.optim import SGD, Adam
from torch.utils.tensorboard import SummaryWriter
import tqdm

import os

save_model_path = "checkpoints/"
pth_name = "saved_model.pth"




def val(model, data_val, loss_function, writer, epoch):
    f1score = 0
    f1 = F1Score(num_classes=3, task = 'multiclass')
    data_iterator = enumerate(data_val)  # take batches
    f1_list = []
    f1t_list = []

    with torch.no_grad():
        model.eval()  # switch model to evaluation mode
        tq = tqdm.tqdm(total=len(data_val))
        tq.set_description('Validation:')

        total_loss = 0

        for _, batch in data_iterator:
            # forward propagation
            image, label = batch
            image = image.cuda()
            label = label.cuda()
            pred = model(image)

            loss = loss_function(pred, label.float())
            loss = loss.cuda()

            pred = pred.softmax(dim=1)
            
            f1_list.extend(torch.argmax(pred, dim =1).tolist())
            f1t_list.extend(torch.argmax(label, dim =1).tolist())
            #f1score += f1_score(label.squeeze().detach().cpu(), pred.squeeze().detach().cpu())

            total_loss += loss.item()
            tq.update(1)

    writer.add_scalar("Validation mIoU", f1score/len(data_val), epoch)
    writer.add_scalar("Validation Loss", total_loss/len(data_val), epoch)


    tq.close()
    print("F1 score: ", f1(torch.tensor(f1_list), torch.tensor(f1t_list)))


    return None


def train(model, dataloader, val_loader, optimizer, loss_fn, n_epochs):
    device = 'cuda'
    writer = SummaryWriter()

    model.cuda()  # Move the model to the specified device (e.g., GPU or CPU)
    model.train()  # Set the model to training mode
    for epoch in range(n_epochs):
        running_loss = 0.0
        tq = tqdm.tqdm(total=len(dataloader))
        tq.set_description('epoch %d' % (epoch))
        f1score = 0

        for i, (images, labels) in enumerate(dataloader):
            images = images.to(device)  # Move the batch of images to the specified device
            labels = labels.to(device)  # Move the batch of labels to the specified device
            
            optimizer.zero_grad()  # Reset the gradients of the optimizer
            
            # Forward pass
            outputs = model(images)
            
            # Compute loss
            loss = loss_fn(outputs, labels)
            outputs = outputs.softmax(dim=1)

            # Backward pass
            loss.backward()
            #f1score += f1_score(labels.detach().cpu(), outputs.detach().cpu())

            # Update model parameters
            optimizer.step()
            
            running_loss += loss.item()
            tq.set_postfix(loss_st='%.6f' % loss.item())
            tq.update(1)
        tq.close()
        epoch_loss = running_loss / len(dataloader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, n_epochs, epoch_loss))
        
        print(f1score/len(dataloader))
        
        
        val(model, val_loader, loss_fn, writer, epoch)
        
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(checkpoint, os.path.join(save_model_path, pth_name))
        print("saved the model " + save_model_path)
        model.train()

train_data = custom_dataset("train")
val_data = custom_dataset("val")

train_loader = DataLoader(
    train_data,
    batch_size=8,
    shuffle=True
)

val_loader = DataLoader(
    val_data,
    batch_size=8
)

model = ExModel(3).cuda()   # Initialsing an object of the class.
#optimizer = SGD(model.parameters(),  lr=0.001)
optimizer = Adam(model.parameters(), lr=0.0005)
loss = nn.CrossEntropyLoss()


# if you want to load your pretrained model or
# you want to resume stopped training
# use torch.load_state_dict by checking the library!


train(model, train_loader, val_loader,  optimizer,loss, 5)

check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']
check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']




  0%|          | 0/239 [00:00<?, ?it/s]

epoch 0:   0%|          | 0/239 [00:00<?, ?it/s]

epoch 0:   0%|          | 0/239 [00:00<?, ?it/s, loss_st=1.282769]

epoch 0:   0%|          | 1/239 [00:00<00:31,  7.50it/s, loss_st=1.282769]

epoch 0:   0%|          | 1/239 [00:00<00:31,  7.50it/s, loss_st=0.590949]

epoch 0:   1%|          | 2/239 [00:00<00:31,  7.55it/s, loss_st=0.590949]

epoch 0:   1%|          | 2/239 [00:00<00:31,  7.55it/s, loss_st=1.365646]

epoch 0:   1%|▏         | 3/239 [00:00<00:28,  8.17it/s, loss_st=1.365646]

epoch 0:   1%|▏         | 3/239 [00:00<00:28,  8.17it/s, loss_st=0.787002]

epoch 0:   2%|▏         | 4/239 [00:00<00:27,  8.62it/s, loss_st=0.787002]

epoch 0:   2%|▏         | 4/239 [00:00<00:27,  8.62it/s, loss_st=0.780193]

epoch 0:   2%|▏         | 5/239 [00:00<00:26,  8.96it/s, loss_st=0.780193]

epoch 0:   2%|▏         | 5/239 [00:00<00:26,  8.96it/s, loss_st=0.530364]

epoch 0:   3%|▎         | 6/239 [00:00<00:25,  9.20it/s, loss_st=0.530364]

epo

Epoch [1/5], Loss: 0.3162
0.0




  0%|          | 0/11 [00:00<?, ?it/s]

Validation::   0%|          | 0/11 [00:00<?, ?it/s]

Validation::  18%|█▊        | 2/11 [00:00<00:00, 19.87it/s]

Validation::  45%|████▌     | 5/11 [00:00<00:00, 22.56it/s]

Validation::  73%|███████▎  | 8/11 [00:00<00:00, 23.14it/s]

Validation:: 100%|██████████| 11/11 [00:00<00:00, 22.26it/s]


F1 score:  tensor(0.9659)
saved the model checkpoints/


...

F1 score:  tensor(0.9773)
saved the model checkpoints/




  0%|          | 0/239 [00:00<?, ?it/s]

epoch 4:   0%|          | 0/239 [00:00<?, ?it/s]

epoch 4:   0%|          | 0/239 [00:00<?, ?it/s, loss_st=0.664190]

epoch 4:   0%|          | 1/239 [00:00<00:36,  6.46it/s, loss_st=0.664190]

epoch 4:   0%|          | 1/239 [00:00<00:36,  6.46it/s, loss_st=0.041863]

epoch 4:   1%|          | 2/239 [00:00<00:33,  7.05it/s, loss_st=0.041863]

epoch 4:   1%|          | 2/239 [00:00<00:33,  7.05it/s, loss_st=0.026018]

epoch 4:   1%|▏         | 3/239 [00:00<00:31,  7.53it/s, loss_st=0.026018]

epoch 4:   1%|▏         | 3/239 [00:00<00:31,  7.53it/s, loss_st=0.075978]

epoch 4:   2%|▏         | 4/239 [00:00<00:29,  7.86it/s, loss_st=0.075978]

epoch 4:   2%|▏         | 4/239 [00:00<00:29,  7.86it/s, loss_st=0.041058]

epoch 4:   2%|▏         | 5/239 [00:00<00:28,  8.24it/s, loss_st=0.041058]

epoch 4:   2%|▏         | 5/239 [00:00<00:28,  8.24it/s, loss_st=0.318277]

epoch 4:   3%|▎         | 6/239 [00:00<00:27,  8.56it/s, loss_st=0.318277]

epo

Epoch [5/5], Loss: 0.0726
0.0




  0%|          | 0/11 [00:00<?, ?it/s]

Validation::   0%|          | 0/11 [00:00<?, ?it/s]

Validation::  18%|█▊        | 2/11 [00:00<00:00, 19.30it/s]

Validation::  45%|████▌     | 5/11 [00:00<00:00, 20.20it/s]

Validation::  73%|███████▎  | 8/11 [00:00<00:00, 20.41it/s]

Validation:: 100%|██████████| 11/11 [00:00<00:00, 20.32it/s]


F1 score:  tensor(0.9886)
saved the model checkpoints/


In [113]:
saved_checkpoint = torch.load(os.path.join(save_model_path, pth_name))

model.load_state_dict(saved_checkpoint['state_dict'])

<All keys matched successfully>

In [114]:

test_data = custom_dataset("test")


test_loader = DataLoader(
    test_data,
    batch_size=8
)


def test(model, data_test, loss_function):
    f1score = 0
    f1 = F1Score(num_classes=3, task = 'multiclass')
    data_iterator = enumerate(data_test)  # take batches
    f1_list = []
    f1t_list = []

    with torch.no_grad():
        model.eval()  # switch model to evaluation mode
        tq = tqdm.tqdm(total=len(data_test))
        tq.set_description('Testing:')

        total_loss = 0

        for _, batch in data_iterator:
            # forward propagation
            image, label = batch
            image = image.cuda()
            label = label.cuda()
            pred = model(image)

            loss = loss_function(pred, label.float())
            loss = loss.cuda()

            pred = pred.softmax(dim=1)
            
            f1_list.extend(torch.argmax(pred, dim =1).tolist())
            f1t_list.extend(torch.argmax(label, dim =1).tolist())
            #f1score += f1_score(label.squeeze().detach().cpu(), pred.squeeze().detach().cpu())

            total_loss += loss.item()
            tq.update(1)

 
    tq.close()
    print("F1 score: ", f1(torch.tensor(f1_list), torch.tensor(f1t_list)))


    return None



test(model, test_loader, loss)

check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']




  0%|          | 0/24 [00:00<?, ?it/s]

Testing::   0%|          | 0/24 [00:00<?, ?it/s]

Testing::  12%|█▎        | 3/24 [00:00<00:01, 20.91it/s]

Testing::  25%|██▌       | 6/24 [00:00<00:00, 21.29it/s]

Testing::  38%|███▊      | 9/24 [00:00<00:00, 21.54it/s]

Testing::  50%|█████     | 12/24 [00:00<00:00, 21.68it/s]

Testing::  62%|██████▎   | 15/24 [00:00<00:00, 22.03it/s]

Testing::  75%|███████▌  | 18/24 [00:00<00:00, 21.99it/s]

Testing::  88%|████████▊ | 21/24 [00:00<00:00, 21.96it/s]

Testing:: 100%|██████████| 24/24 [00:01<00:00, 21.80it/s]

F1 score:  tensor(0.9948)
